In [1]:
import os
import uuid

import pandas as pd
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, PointStruct, VectorParams
from sentence_transformers import SentenceTransformer

load_dotenv()

%load_ext autoreload
%autoreload 2

c:\Users\S\PycharmProjects\SimilarBooksRecommendation\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Загрузка датасета

In [2]:
df_1 = pd.read_csv(os.path.join(os.getcwd(), "data", "комиксы.csv"), on_bad_lines="skip").dropna()
df_2 = pd.read_csv(os.path.join(os.getcwd(), "data", "средневековье.csv"), on_bad_lines="skip").dropna()
df_3 = pd.read_csv(os.path.join(os.getcwd(), "data", "художественная.csv"), on_bad_lines="skip").dropna()

df = pd.concat([df_1, df_2, df_3], axis=0).drop_duplicates(subset=["Title", "Author"]).drop_duplicates(subset=["Title"])
df = df[df["Description"].apply(len) >= 100]

df["ID"] = [uuid.uuid5(uuid.NAMESPACE_DNS, title).hex for title in df["Title"]]
# df['Title'] = df['Title'].apply(lambda x: x.lower().capitalize())

In [3]:
print(df.shape)
display(df.head())

(16149, 8)


,Title,Author,Link,Image,Category,Description,Info,ID
0,Истории книжных магазинов,Ивашкина М.,https://www.podpisnie.ru/books/istorii-knizhny...,/upload/resize_cache/iblock/cfe/160_230_1/8vg4...,Графические романы / Комиксы,Книжный магазин — место где книга и человек на...,Автор Ивашкина М. Издательство Миля Год издани...,e26498c9c4a75252a18a1c677b6d3ae9
1,Полуночники,Эвенс Б.,https://www.podpisnie.ru/books/polunochniki/,/upload/resize_cache/iblock/a5b/160_230_1/tj5v...,Графические романы / Комиксы,Одна ночь. Три незнакомца. Три совершенно разн...,Автор Эвенс Б. Издательство Бумкнига Год издан...,2d4a036599e959a0a9e76e32a2075200
2,Госпожа Кагуя: В любви как на войне. Любовная...,Акасака А.,https://www.podpisnie.ru/books/gospozha-kaguya...,/upload/resize_cache/iblock/7fd/160_230_1/uqfn...,Графические романы / Комиксы,Завещание Ганъана Синомии уничтожено и Кагуя в...,Автор Акасака А. Издательство Азбука Год издан...,9ac06e3bafd25975bbf6fb9083e18567
3,"Люди, которые легко становятся счастливыми. ...",Дэнсинг С.,https://www.podpisnie.ru/books/lyudi-kotorye-l...,/upload/resize_cache/iblock/929/160_230_1/y4yj...,Графические романы / Комиксы,Привычная рутина отбирает краски жизни а о «ма...,Автор Дэнсинг С. Издательство КоЛибри Год изда...,ca68d65336425ee49bb1e563fb832e1f
4,"Моя геройская академия. Кн. 19. Те, кто об...",Хорикоси К.,https://www.podpisnie.ru/books/moya-geroyskaya...,/upload/resize_cache/iblock/f6d/160_230_1/7hh3...,Графические романы / Комиксы,Решающее сражение за судьбу мира в самом разга...,Автор Хорикоси К. Издательство Азбука Год изда...,2cb378c590605ddb8c4cf85b8ee1518f


In [4]:
# import joblib
# hash_map = {uid: title for uid, title in zip(df["ID"], df["Title"])}
# joblib.dump(hash_map, os.path.join(os.getcwd(), "backend", "config", "title_id_hash_map.pkl"))

# Загрузка моделей и подключений

In [4]:
embedder = SentenceTransformer(os.getenv("QDRANT_EMBEDDER"))
qdrant = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))

# Пересоздать коллекцию Qdrant

In [ ]:
# vectors = embedder.encode(df["Description"].values, batch_size=1, normalize_embeddings=True, show_progress_bar=True)

In [ ]:
# qdrant.recreate_collection(
#     collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
#     vectors_config=VectorParams(size=312, distance=Distance.COSINE),
# )

In [ ]:
# from tqdm import tqdm

# # Параметры для пакетной обработки
# batch_size = 300  # Размер пакета, можно настроить по мере необходимости
# points = []

# # Обрабатываем данные по пакетам
# for uid, category, author, title, image_link, vector in tqdm(
#     zip(df["ID"], df["Category"], df["Author"], df["Title"], df["Image"], vectors)
# ):
#     points.append(
#         PointStruct(
#             id=uid,
#             payload={
#                 "category": category,
#                 "author": author,
#                 "title": title,
#                 "image_link": image_link,
#             },
#             vector=vector,
#         )
#     )

#     # Если пакет набран, отправляем его в Qdrant
#     if len(points) >= batch_size:
#         qdrant.upsert(collection_name=os.getenv("QDRANT_COLLECTION_NAME"), points=points)
#         points = []  # Очищаем список для следующего пакета

# # Отправляем оставшиеся точки, если они есть
# if points:
#     qdrant.upsert(collection_name=os.getenv("QDRANT_COLLECTION_NAME"), points=points)

# Пример запроса в Qdrant

In [9]:
title = "Токийский гуль: re.  квест"

In [10]:
query_vector = embedder.encode(title)
n_chunks = 5
offset = 0

In [11]:
content = qdrant.search(
    collection_name=os.getenv("QDRANT_COLLECTION_DESCRIPTION"), query_vector=query_vector, limit=n_chunks, offset=offset
)
content

[ScoredPoint(id='3c2095f8-91fa-5a8d-92ca-9f30c407831d', version=4, score=0.82746696, payload={'author': 'Исида С.', 'category': 'Графические романы / Комиксы', 'image_link': '/upload/resize_cache/iblock/223/160_230_1/fzhe2zm0u65q5yislcrbrgg78tb6xz6j.jpg', 'title': 'Токийский гуль: re.  Кн.  2'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='71a2f3d6-abdf-5070-a4fc-abd7c43edde3', version=35, score=0.7933022, payload={'author': 'Чассапакис Д.', 'category': 'Художественная литература', 'image_link': '/upload/resize_cache/iblock/354/160_230_1/fzwhqmn79dsr1r3f4h284o05vzj33xja.jpg', 'title': 'Дневник 29.  Забвение'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='96a658a4-2d48-5a1a-9725-bc924ff05e12', version=35, score=0.792939, payload={'author': 'Хэйс К.', 'category': 'Художественная литература', 'image_link': '/upload/resize_cache/iblock/2fe/160_230_1/j13fp610kavh72qzvmcqsw4r9zlwiq6u.jpg', 'title': 'Затворники'}, vector=None, shard_key=None, order_valu

# Redis

In [73]:
import os

import redis

# Подключение к Redis
redis_client = redis.StrictRedis(
    host=os.getenv("REDIS_HOST"), port=os.getenv("REDIS_PORT"), password=os.getenv("RESID_PASSWORD")
)

try:
    # Проверка подключения к Redis
    pong = redis_client.ping()
    if pong:
        print("Подключение к Redis успешно!")
    else:
        print("Не удалось подключиться к Redis.")
except redis.ConnectionError as e:
    print(f"Ошибка подключения к Redis: {e}")

Подключение к Redis успешно!


## Быстрая загрузка данных в redis

In [78]:
# Очистка всех данных
redis_client.flushall()

True

In [79]:
from tqdm import tqdm

# Преобразование данных в список кортежей для удобства использования
data = list(zip(df["Title"], df["Description"], df["Category"], df["Author"], df["Image"], df["Info"], df["ID"]))
# Определение размера батча
batch_size = 1000

# Использование пайплайна для записи данных и метаданных
for i in tqdm(range(0, len(data), batch_size)):
    with redis_client.pipeline() as pipe:
        for title, description, category, author, image, info, uid in data[i : i + batch_size]:
            # Установка данных
            pipe.set(title, description)

            # Установка метаданных
            metadata_key = f"{title}:metadata"
            pipe.hset(metadata_key, "category", category)
            pipe.hset(metadata_key, "author", author)
            pipe.hset(metadata_key, "image", image)
            pipe.hset(metadata_key, "info", info)
            pipe.hset(metadata_key, "uid", uid)

        # Выполнение всех команд в пайплайне
        pipe.execute()

100%|██████████| 17/17 [00:16<00:00,  1.03it/s]


In [16]:
num_keys = redis_client.dbsize()

print(f"Number of keys in the database: {num_keys}")

Number of keys in the database: 32298


# Создание коллекции с названиями книги

In [17]:
qdrant.recreate_collection(
    collection_name="BookTitles",
    vectors_config=VectorParams(size=312, distance=Distance.DOT),
)

vectors = embedder.encode(df["Title"].values, batch_size=16, normalize_embeddings=True, show_progress_bar=True)

from tqdm import tqdm

# Параметры для пакетной обработки
batch_size = 300  # Размер пакета, можно настроить по мере необходимости
points = []

# Обрабатываем данные по пакетам
for uid, title, vector in tqdm(zip(df["ID"], df["Title"], vectors)):
    points.append(
        PointStruct(
            id=uid,
            payload={
                "title": title,
            },
            vector=vector,
        )
    )

    # Если пакет набран, отправляем его в Qdrant
    if len(points) >= batch_size:
        qdrant.upsert(collection_name="BookTitles", points=points)
        points = []  # Очищаем список для следующего пакета

# Отправляем оставшиеся точки, если они есть
if points:
    qdrant.upsert(collection_name="BookTitles", points=points)

C:\Users\S\AppData\Local\Temp\ipykernel_25188\3831286106.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

# Запрос с фильтром

In [9]:
# from qdrant_client.http import models


# def filter_search(query, collection_name, limit=6, offset=0, title_filter=None):
#     # Создание векторного запроса
#     query_vector = embedder.encode(
#         query,
#         batch_size=1,
#         normalize_embeddings=True,
#     )

#     # Построение условий фильтрации
#     filter_conditions = []
#     if title_filter:
#         filter_conditions.append(models.FieldCondition(key="title", match=models.MatchValue(value=title_filter)))

#     # Выполнение поиска
#     results = qdrant.search(
#         collection_name=collection_name,
#         query_vector=query_vector,
#         query_filter=models.Filter(must=filter_conditions) if filter_conditions else None,
#         limit=limit,
#         offset=offset,
#     )

#     return results

In [14]:
# # Пример запроса
# search_results = filter_search(
#     query=query,
#     collection_name="BookTitles",
#     limit=n_chunks,
#     # category_filter=category,
#     # author_filter=author,
#     title_filter=query,
# )

# for result in search_results:
#     print(f"Title: {result.payload['title']}, Score: {result.score}")

Title: Ведьмак, Score: 1.0


# Автокомплит

In [24]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False


class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search_prefix(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return []
            node = node.children[char]
        return self._collect_all_words(node, prefix)

    def _collect_all_words(self, node, prefix):
        words = []
        if node.is_end_of_word:
            words.append(prefix)
        for char, child_node in node.children.items():
            words.extend(self._collect_all_words(child_node, prefix + char))
        return words


# Функция автокомплита
def autocomplete_books_trie(prefix, trie, limit=5):
    suggestions = trie.search_prefix(prefix)  # .lower()
    return suggestions[:limit]


# Инициализация Trie и вставка книг
trie = Trie()

books = df["Title"]

for book in books:
    trie.insert(book)  # .lower()

In [25]:
# Пример использования
user_input = "Токийский"
suggestions = autocomplete_books_trie(user_input, trie, limit=10)
print(suggestions)
# print(list(map(str.capitalize, suggestions)))

['Токийский гуль: re.  Квест', 'Токийский гуль: re.  Кн.  8', 'Токийский гуль: re.  Кн.  7', 'Токийский гуль: re.  Кн.  2', 'Токийский гуль: re.  Кн.  1', 'Токийский гуль: re.  Книга 6', 'Токийский гуль: re.  Книга 5', 'Токийский гуль: re.  Книга 3', 'Токийский гуль: re.  Книга 4', 'Токийский гуль: zakki: re']


In [26]:
import joblib

joblib.dump(trie, os.path.join(os.getcwd(), "data", "trie.pkl"))

['c:\\Users\\S\\PycharmProjects\\SimilarBooksRecommendation\\data\\trie.pkl']

In [27]:
import os

import joblib

trie = joblib.load(os.path.join(os.getcwd(), "data", "trie.pkl"))

user_input = "Токийский"
suggestions = autocomplete_books_trie(user_input, trie, limit=10)
print(suggestions)


['Токийский гуль: re.  Квест', 'Токийский гуль: re.  Кн.  8', 'Токийский гуль: re.  Кн.  7', 'Токийский гуль: re.  Кн.  2', 'Токийский гуль: re.  Кн.  1', 'Токийский гуль: re.  Книга 6', 'Токийский гуль: re.  Книга 5', 'Токийский гуль: re.  Книга 3', 'Токийский гуль: re.  Книга 4', 'Токийский гуль: zakki: re']


# PostgreSQL

In [4]:
import os

import pandas as pd
from sqlalchemy import create_engine

# Получаем креды из переменных окружения

host = os.getenv("POSTGRE_HOST")

port = os.getenv("POSTGRE_PORT")

user = os.getenv("POSTGRE_USER")

password = os.getenv("POSTGRE_PASSWORD")

database = os.getenv("POSTGRE_DATABASE")

users_table_name = os.getenv("POSTGRE_USER_TABLE")

books_table_name = os.getenv("POSTGRE_BOOK_TABLE")


In [5]:
df.tail()

,Title,Author,Link,Image,Category,Description,Info,ID
18690,Дитя божье,Маккарти К.,https://www.podpisnie.ru/books/ditya-bozh-e/,/upload/no-image.png,Художественная литература,США самое начало подернутых дымкой ностальгии ...,Автор Маккарти К. Издательство Найди лесоруба ...,a0faa7ca602e50778f6bb6ba9dabe50c
18691,Говорящая свеча,Бротиган Р.,https://www.podpisnie.ru/books/govoryashchaya-...,/upload/no-image.png,Художественная литература,Ричард Бротиган (1935-1984) — американский про...,Автор Бротиган Р. Издательство Найди лесоруба ...,bcdd85caccfa5646810dfe3bbdc1df2c
18692,"Сад, где живут кентавры",Гелианов А.,https://www.podpisnie.ru/books/sad-gde-zhivut-...,/upload/no-image.png,Художественная литература,Исчез ли модернизм как жанр к 2024 году? Или с...,Автор Гелианов А. Издательство Найди лесоруба ...,25dee3a018a0547793ce48babc5a4426
18753,Бабочка во сне: сборник стихотворений,Сонги Л.,https://www.podpisnie.ru/books/babochki-vo-sne...,/upload/no-image.png,Художественная литература,Поэт Лим Сонги родился в 1968 г. в городе Инчх...,Автор Сонги Л. Издательство Гиперион. Год изда...,53434f762cf051e7a0bdefd7b653e3f5
18952,Чапыгин Т1-5,Чапыгин А. П.,https://www.podpisnie.ru/books/chapygin-t1-5/,/upload/no-image.png,Художественная литература,Алексей Павлович Чапыгин - один из основополож...,Автор Чапыгин А. П. Год издания 2011 Артикул 1...,399f7b1629355ed0ba411276e341d480


In [8]:
# Подключаемся к базе данных через SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Создаем таблицу в базе данных из DataFrame
df.to_sql(books_table_name, engine, if_exists="replace", index=False)

# Закрываем соединение
engine.dispose()


In [7]:
from backend.src.schemas.tamplates import BookInfo, BooksBatchResponse


from typing import Optional
from sqlalchemy.engine import Engine
import pandas as pd

def get_books_batch(engine: Engine, limit: int, offset: int, category_filter: Optional[str] = None) -> BooksBatchResponse:
    main_query = """
    SELECT *
    FROM books_table
    """
    filter_query = f"""WHERE "Category" = '{category_filter}'""" if category_filter else ""

    limit_query = f"""LIMIT {limit} OFFSET {offset};"""
    query = main_query + filter_query + limit_query

    df = pd.read_sql(query, engine)

    # Преобразование DataFrame в список словарей
    books_list = df.to_dict(orient='records')

    # Создание экземпляров Pydantic классов
    books_info_list = [BookInfo(**book) for book in books_list]

    # Создание экземпляра BooksBatch
    books_batch = BooksBatchResponse(books=books_info_list)

    return books_batch


ModuleNotFoundError: No module named 'backend.src.schemas.tamplates'

In [ ]:
limit = 3
offset = 1
# category_filter = "Художественная литература"

In [ ]:
get_books_batch(engine, limit, offset)

BooksBatch(books=[BookInfo(category='Графические романы / Комиксы', author='Эвенс Б.', image='/upload/resize_cache/iblock/a5b/160_230_1/tj5vuz4wqf2q98uzmueav5fjwvabwqj0.jpg', info='Автор Эвенс Б. Издательство Бумкнига Год издания 2023 Переплет Твёрдый Страниц 336 Формат 205х250 мм Язык Русский ISBN 978-5-907305-19-9 Артикул 1135825', uid='2d4a036599e959a0a9e76e32a2075200', description='Одна ночь. Три незнакомца. Три совершенно разных человека вынужденных скрывать своё истинное Я под маской благоразумия. Случайно оказавшись в одном месте в одно время они отчаянно жаждут свободы и развлечений. Под манящими и безжалостными огнями ночного города их истории переплетаются балансируя на грани чудесного сна и невыносимого кошмара. Брехт Эвенс — современный фламандский автор комиксов и иллюстратор которого газета The Guardian назвала одним из самых талантливых представителей бельгийской школы иллюстрации со времён Эрже автора комиксов про Тинтина. За графический роман «Полуночники» Брехт был уд